In [97]:
os.environ["OPENAI_API_KEY"] = "sk-"

#RunnableParallel is a class designed to run multiple tasks or chains in parallel. This means it can execute several functions, prompts, or actions at the same time rather than one after the other, which can significantly speed up workflows that don’t rely on each other’s results.

#RunnableParallel

In [98]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
import os

model = ChatOpenAI(model="gpt-4o-mini")

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "marriage"})


{'joke': AIMessage(content='Why do married people live longer?\n\nBecause they can’t argue with their spouse if they’re dead!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-01b591ba-d562-454d-82de-5ff5d2814163-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 'poem': AIMessage(content='Two hearts entwined, in laughter and tears,  \nA journey of love that conquers all fears.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens':

#RunnablePasthrough

In [99]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retrieval_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke({"question":"what is praveen full name?","context":"Praveen full name is Praveen Reddy C and he is from Munagala Village, He loves his village, His father is farmer who cultivates fruits and palm oil"})

"Praveen's full name is Praveen Reddy C."

In [100]:

retrieval_chain.invoke({"question":"what is praveen full name and his village name?","context":"Praveen full name is Praveen Reddy C and he is from Munagala Village, He loves his village, His father is farmer who cultivates fruits and palm oil"})

"Praveen's full name is Praveen Reddy C and his village name is Munagala Village."

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages( 
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

model = ChatOpenAI(temperature=0)

runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


#When you use .bind(stop="SOLUTION"), you’re telling the model that, every time it generates output, it should stop when it reaches the word "SOLUTION"

#In short, .bind(stop="SOLUTION") ensures that the OpenAI model knows when to stop generating text (after the solution is reached), making the output more predictable and structured.

In [24]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




#Custom Function 

#in which we are passing argument from one step to next step 

In [32]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(d:dict):
    length  = 0
    for k,v in d.items():
        length = length + len(v)
    return length


model = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("Writing a Love Letter by a {a} to {b} with {wrd} words")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("name1"),   # Extract name1 from input dict
        "b": itemgetter("name2"),   # Extract name2 from input dict
        "wrd": RunnableLambda(lambda x: length_function({"name1": x["name1"], "name2": x["name2"]}))  
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"name1": "Praveen", "name2": "Sahasra"})

'My dearest Sahasra, you fill my heart with joy and love every single day.'

##Create a chain with a python method !!

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain

prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")


@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})


custom_chain.invoke("marriage")

'The subject of this joke is scarecrows and successful marriages.'

#runtime chain internals

##configurable_fields method

This allows you to adjust certain settings (like temperature or parameters) for a specific task or step in a process, while the process is running.

Think of it as giving you control over individual steps in a chain, without needing to decide everything in advance (like with .bind()).

Example: 
If you're generating text using an AI model, you can change how "creative" (temperature) the AI is during runtime, rather than locking in this value before the process starts.

-------------

##configurable_alternatives method:

This allows you to swap out different methods or models during runtime.

You can list multiple alternative options (for example, different AI models or ways of solving a problem) and choose which one to use when the process runs.

Example: 
If you have two AI models (like GPT-3 and another model), you can switch between them as needed without changing your code.

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

model.invoke("Write a Love Letter with 15 words")

AIMessage(content='My dearest love, you are my everything, my heart beats only for you. I love you.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-35ea896e-37a0-40aa-a93b-01c097f2c36a-0', usage_metadata={'input_tokens': 15, 'output_tokens': 21, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [43]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("Write a Love Letter with 15 words")

AIMessage(content='My darling, you are my light, my heart, my everything. I love you endlessly and forever.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-54c78160-c797-497a-a6af-b10842d07bc2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 21, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [44]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

chain.invoke({"x": 100})

AIMessage(content='235', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a382466-c544-4bad-aa59-14d5ec3db4ee-0', usage_metadata={'input_tokens': 14, 'output_tokens': 1, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="anthropic",
    openai=ChatOpenAI(),
    gpt4=ChatOpenAI(model="gpt-4"),
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

# By default it will call Anthropic
chain..with_config(configurable={"id": "llm"})invoke({"topic": "Love"})

In [56]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = PromptTemplate.from_template(
    "Write A 3 liner love story on my Wife {name}"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="love",
    poem=PromptTemplate.from_template("Write A 3 line love poem to my Love {name}"),
)
chain = prompt | llm | StrOutputParser()


In [57]:
print(chain.invoke({"name": "Sahasra"}))

In a bustling café, Sahasra's laughter danced through the air, capturing my heart like a melody. With every shared glance, our souls intertwined, painting a love story only we could understand. Together, we built a world where every moment felt like a beautiful forever.


In [60]:
print(chain.with_config(configurable={"prompt": "poem"}).invoke({"name": "Naveena"}))

In the garden of dreams, your laughter blooms bright,  
Naveena, my heart dances in your gentle light,  
Together we weave a tapestry of love, pure and right.


In [101]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Glue`, `Databricks`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
)

chain = prompt | llm | StrOutputParser()


chain.invoke({"question": "how do I cuse AWS Glue ?"})

'Glue'

In [85]:
glue_chain = PromptTemplate.from_template(
    """You are an expert in AWS ETL Service Glue. \
Always answer questions starting with "As the AWS Certified Praveen Reddy told me". \
Respond to the following question in 2 lines:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0)


databricks_chain = PromptTemplate.from_template(
    """You are an expert in Databricks. \
Always answer questions starting with "As Legendary Databricks Expert Chinnareddy once told ". \
Respond to the following question in 2 lines: 

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0)


general_chain = PromptTemplate.from_template(
    """Respond to the following question in 2 lines:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [86]:
def route(info):
    if "glue" in info["topic"].lower():
        return glue_chain
    elif "databricks" in info["topic"].lower():
        return databricks_chain
    else:
        return general_chain

In [87]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [102]:
chain.invoke({"question": "How to Write a Glue Catalog in Glue?"})

'Glue'

In [103]:
full_chain.invoke({"question": "How to Write a Glue Catalog in Glue?"})

AIMessage(content='As the AWS Certified Praveen Reddy told me, you can write a Glue Catalog in Glue by using the AWS Glue Console or the AWS Glue API to create and manage tables and databases. Additionally, you can use Glue Crawlers to automatically discover and catalog your data sources.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 59, 'total_tokens': 115, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-3a42de12-7340-4526-ba02-f1a84edd5078-0', usage_metadata={'input_tokens': 59, 'output_tokens': 56, 'total_tokens': 115, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [104]:
chain.invoke({"question": "What is Unity Catalog in Databricks"})

'Databricks'

In [105]:
full_chain.invoke({"question": "What is Unity Catalog in Databricks"})

AIMessage(content='As Legendary Databricks Expert Chinnareddy once told, Unity Catalog is a unified governance solution for all data assets in Databricks, enabling fine-grained access control and data discovery. It simplifies data management across various workspaces and provides a centralized view of data assets.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 59, 'total_tokens': 115, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b973a80-b306-4e7e-a323-dfb22a3e01d3-0', usage_metadata={'input_tokens': 59, 'output_tokens': 56, 'total_tokens': 115, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

#Inspect Runnables 

In [96]:
full_chain.get_graph().print_ascii()

                    +-------------------------------+                      
                    | Parallel<topic,question>Input |                      
                    +-------------------------------+                      
                             ***           ***                             
                           **                 **                           
                         **                     **                         
              +----------------+                  **                       
              | PromptTemplate |                   *                       
              +----------------+                   *                       
                       *                           *                       
                       *                           *                       
                       *                           *                       
                +------------+                     *                       
            